In [ ]:
import random
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from math import exp
%matplotlib inline

In [ ]:
num_of_layers = int(input('Enter the number of layers: '))

In [ ]:
num_of_neurons = int(input('Enter the number of neurons: '))

In [ ]:
def initialize_network(n, l, h, o):
    def r():
        return random.uniform(-0.50, 0.50)
    
    neural_network = []
    for i in range(l):
        if i == 0:
            neural_network.append([{'w':[r() for i in range(n+1)]} for j in range(h)])
        else:
            neural_network.append([{'w':[r() for i in range(h+1)]} for j in range(h)])
    
    neural_network.append([{'w':[r() for i in range(h+1)]} for j in range(o)])
    return neural_network

In [ ]:
def summing_function(weights, inputs):
    bias = weights[-1]
    summ = 0.00
    for i in range(len(weights)-1):
        summ += (weights[i] * float(inputs[i]))
    return summ + bias

In [ ]:
def activation_function(z):
    return 1 / (1 + exp(-z))

In [ ]:
def feed_forward(network, example):
    layer_input, layer_output = example, []
    for layer in network:
        for neuron in layer:
            summ = summing_function(neuron['w'], layer_input)
            neuron['o'] = activation_function(summ)
            layer_output.append(neuron['o'])
        layer_input, layer_output = layer_output, []
    return layer_input

In [ ]:
def activation_derivative(z):
    return z * (1 - z)

In [ ]:
def backpropagate(network, example):
    for i in range(len(network)-1, -1, -1):
        for j in range(len(network[i])):
            err = 0.00
            if i == len(network)-1:
                err = example[j] - network[i][j]['o']
            else:
                summ = 0.00
                for neuron in network[i+1]:
                    summ += neuron['w'][j] * neuron['d']
                err = summ
            network[i][j]['d'] = activation_derivative(network[i][j]['o']) * err

In [ ]:
def update_weights(network, example, delta):
    for i in range(len(network)):
        if i != 0:
            t = [neuron['o'] for neuron in network[i-1]]
        else:
            t = example[:-1]
        for neuron, d in zip(network[i], range(0, len(network[i]))):
            for f in range(len(t)):
                neuron['w'][f] += LEARNING_RATE * float(t[f]) * neuron['d']
                if delta is not None:
                    neuron['w'][f] += MOMENTUM_RATE * delta[d]
                neuron['w'][-1] += LEARNING_RATE * neuron['d']

In [ ]:
def sse(actual, target):
    summ = 0.00
    for i in range(len(actual)):
        summ += (actual[i] - target[i])**2
    return summ

In [ ]:
def stochastic_gradient_descent(network, classes, training_data):
    for epoch in range(0, EPOCHS):
        first_example = True
        total_error = 0.00
        for example in training_data:
            temporal_delta = [neuron['d'] \
                for layer in network for neuron in layer] \
                if not first_example else None
            outputs = [0 for _ in range(classes)]
            outputs[int(example[-1])] = 1
            actual = feed_forward(network, example)
            total_error += sse(actual, outputs)
            backpropagate(network, outputs)
            update_weights(network, example, temporal_delta)
            reset_neurons(network)
            first_example = False
        print('>epoch=%d, error=%.3f' %(epoch, total_error))
        #if epoch % 10 == 0:
        MSE.append(total_error/len(training_data))
        #TRP.append(performance_measure(NETWORK, TRAIN))
        #TEP.append(performance_measure(NETWORK, TEST))

In [ ]:
def reset_neurons(network):
    for layer in network:
        for neuron in layer:
            neuron['o'] = 0

In [ ]:
def performance_measure(network, data):
    correct, total = 0, 0
    for example in data:
        if check_output(network, example) == float(example[-1]):
            correct += 1
        total += 1
    return 100*(correct / total)

def check_output(network, example):
    output = feed_forward(network, example)
    return output.index(max(output))


In [ ]:
def plot_data():
    x = range(0, EPOCHS)
    fig, ax2 = plt.subplots()
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('MSE', color='blue')
    line, = ax2.plot(x, MSE, '-', c='blue', lw='1', label='MSE')
    fig.tight_layout()
    plt.show()
    plt.clf()

In [ ]:
data = pd.read_csv('MNISTnumImages5000_balanced.txt', sep = '\t', header=None)
data['labels'] = pd.read_csv('MNISTnumLabels5000_balanced.txt', header=None)

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
train_data.reset_index(inplace=True, drop=True)
train_data = train_data.values.tolist()
for i in range(len(train_data)):
    train_data[i][-1] = int(train_data[i][-1])

test_data.reset_index(inplace=True, drop=True)
test_data = test_data.values.tolist()
for i in range(len(test_data)):
    test_data[i][-1] = int(test_data[i][-1])

In [ ]:
TRAIN = train_data
features = len(TRAIN[0][:-1])
classes = len(list(set([c[-1] for c in TRAIN])))
NETWORK = initialize_network(features, num_of_layers, num_of_neurons, classes)
LEARNING_RATE, MOMENTUM_RATE = 0.03, 0.5

EPOCHS = 100
MSE, TRP, TEP = [], [], []
stochastic_gradient_descent(NETWORK, classes, TRAIN)


In [ ]:
result = []
for row in train_data:
    prediction = check_output(NETWORK, row)
    result.append([row[-1],prediction])

In [ ]:
cm_train = np.zeros((10,10))
for i in range(len(result)):
    cm_train[result[i][0]][result[i][1]] = cm_train[result[i][0]][result[i][1]] + 1

In [ ]:
cm_train

In [ ]:
cm_df = pd.DataFrame(cm_train)
cm_df.to_excel('cm_df.xlsx')

In [ ]:
result = []
for row in test_data:
    prediction = check_output(NETWORK, row)
    result.append([row[-1],prediction])

In [ ]:
cm_test = np.zeros((10,10))
for i in range(len(result)):
    cm_test[result[i][0]][result[i][1]] = cm_test[result[i][0]][result[i][1]] + 1

In [ ]:
cm_test

In [ ]:
cm_df_t = pd.DataFrame(cm_test)
cm_df_t.to_excel('cm_df_t.xlsx')

In [ ]:
plot_data()
plt.style.use('default')

In [ ]:
neurons_20 = []
for k in range(20):
    weights = NETWORK[0][k]['w'][:-1]
    n_weights = []
    for l in range(len(weights)):
        n_weights.append((weights[l] - min(weights))/(max(weights) - min(weights)))
    neurons_20.append(n_weights)

In [ ]:
fig, axs = plt.subplots(4, 5,figsize=(10,10))
plt.style.use('grayscale')
j = 0
k = 0
for i in range(len(neurons_20)):
    im = np.reshape(neurons_20[i],(28,28))
    if (i % 5 == 0) and (i != 0):
        j = j + 1  
    axs[j, k].imshow(im)
    axs[j, k].set_title(str(i+1))
    k = k + 1
    if k == 5:
        k = 0  